### Imports


In [1]:
import os
import shutil
# import gzip

import nibabel as nib
import pandas as pd
from sklearn.impute import SimpleImputer

## Merge Both Data Types (Assesments + DaTscan)

### for csv 

In [ ]:
# Load both CSV files
assessments_df = pd.read_csv("Dataset/CSV/assesments_data.csv")
DaTscan_df = pd.read_csv("Dataset/datscan_images/DaTscan_dataset.csv")


# Remove corrupted image entries
corrupted_entries = [
    # failed in Conversion
    {"Image Data ID": "I770243",  "Subject": 3107},
    {"Image Data ID": "I1317564", "Subject": 3179},
    # failed in Denoising
    {"Image Data ID": "I1317554", "Subject": 3372},
    {"Image Data ID": "I449046",  "Subject": 4092},
    {"Image Data ID": "I1273880", "Subject": 50028},
    # failed in cropping
    {"Image Data ID": "I1474392", "Subject": 101281},
    {"Image Data ID": "I1317553", "Subject": 3078},
    {"Image Data ID": "I1317557", "Subject": 3378},
    {"Image Data ID": "I770244",  "Subject": 3500},
    {"Image Data ID": "I449248",  "Subject": 3863}
]

for entry in corrupted_entries:
    DaTscan_df = DaTscan_df[~(
        (DaTscan_df["Image Data ID"] == entry["Image Data ID"]) &
        (DaTscan_df["Subject"] == entry["Subject"])
    )]


# Rename columns in DaTscan to match those in assessments
DaTscan_df = DaTscan_df.rename(columns={
    "Subject": "PATNO",
    "Visit": "EVENT_ID"
})


# merge with indicator
merged_all = pd.merge(
    assessments_df,
    DaTscan_df[["PATNO", "EVENT_ID", "Image Data ID"]],
    on=["PATNO", "EVENT_ID"],
    how="left",
    indicator=True
)


# Split paired / unpaired
paired_df = merged_all[merged_all["_merge"] == "both"].drop(columns=["_merge"])
unpaired_df = merged_all[merged_all["_merge"] == "left_only"].drop(
    columns=["_merge", "Image Data ID"]
)


# Move "Image Data ID" to be 3rd column (in paired csv only)
if "Image Data ID" in paired_df.columns:
    cols = paired_df.columns.tolist()
    cols.remove("Image Data ID")
    cols.insert(2, "Image Data ID")
    paired_df = paired_df[cols]


# Save CSVs
paired_csv_path = "Dataset/CSV/bm3d_striatum/paired_data_datscan.csv"
unpaired_csv_path = "Dataset/CSV/bm3d_striatum/unpaired_data_no_datscan.csv"

paired_df.to_csv(paired_csv_path, index=False)
unpaired_df.to_csv(unpaired_csv_path, index=False)


print(f"Original assessments dataset: {assessments_df.shape}")
print(f"Paired (with DaTscan):        {paired_df.shape}")
print(f"Unpaired (no DaTscan):        {unpaired_df.shape}")

Original assessments dataset: (4097, 84)
Paired (with DaTscan):        (986, 85)
Unpaired (no DaTscan):        (3111, 84)


### for DaTscan folder

In [ ]:
images_dir = "Dataset/datscan_images/Converted_NIfTI"

paired_dir = "Dataset/datscan_images/Converted_NIfTI_paired"
unpaired_dir = "Dataset/datscan_images/Converted_NIfTI_unpaired"

os.makedirs(paired_dir, exist_ok=True)
os.makedirs(unpaired_dir, exist_ok=True)

# load csv
merged_df = pd.read_csv(paired_csv_path)

# BUILD MAPPING
# PATNO (str) -> set of image IDs (str)
patient_to_images = (
    merged_df.groupby("PATNO")["Image Data ID"]
    .apply(lambda x: set(x.astype(str)))
    .to_dict()
)

paired_count = 0
unpaired_count = 0

# MAIN LOOP
for patient_folder in os.listdir(images_dir):
    patient_path = os.path.join(images_dir, patient_folder)

    if not os.path.isdir(patient_path):
        continue

    patient_id = str(patient_folder)

    for file_name in os.listdir(patient_path):
        if not file_name.endswith(".nii.gz"):
            continue

        image_id = file_name.replace(".nii.gz", "")
        src_path = os.path.join(patient_path, file_name)

        # CASE 1: paired image
        if (
            patient_id in map(str, patient_to_images.keys()) and
            image_id in patient_to_images[int(patient_id)]
        ):
            dst_patient_dir = os.path.join(paired_dir, patient_id)
            os.makedirs(dst_patient_dir, exist_ok=True)

            shutil.copy2(
                src_path,
                os.path.join(dst_patient_dir, file_name)
            )
            paired_count += 1

        # CASE 2: unpaired image
        else:
            dst_patient_dir = os.path.join(unpaired_dir, patient_id)
            os.makedirs(dst_patient_dir, exist_ok=True)

            shutil.copy2(
                src_path,
                os.path.join(dst_patient_dir, file_name)
            )
            unpaired_count += 1


# FINAL SUMMARY
print(f"✅ Image dataset ({images_dir}) successfully split")
print(f"Paired images copied:    {paired_count}")
print(f"Unpaired images copied:  {unpaired_count}")

✅ Image dataset successfully split
Paired images copied:    986
Unpaired images copied:  2848


### Verification

#### Patient-level consistency

In [6]:
csv_path = "Dataset/CSV/bm3d_striatum/paired_data_datscan.csv"
images_root = "Dataset/datscan_images/bm3d_striatum/Cropped_NIfTI_paired"

# Get patient IDs from CSV
df = pd.read_csv(csv_path)
csv_patients = set(df['PATNO'].astype(str).unique())

# Get patient IDs from image folders
folder_patients = set(os.listdir(images_root))

# Compare
only_in_csv = csv_patients - folder_patients
only_in_folder = folder_patients - csv_patients
both = csv_patients & folder_patients

print("Patients only in CSV:", len(only_in_csv))
print("Patients only in Folder:", len(only_in_folder))

if only_in_csv:
    print("Missing in images:", only_in_csv)
if only_in_folder:
    print("Missing in CSV:", only_in_folder)

# Count images for patients in both
total_images = 0
for patient_id in both:
    patient_folder = os.path.join(images_root, patient_id)
    total_images += len([f for f in os.listdir(patient_folder) if f.endswith(".nii") or f.endswith(".nii.gz")])


print("\nSummary:")
print(f"Total patients in CSV: {len(csv_patients)}")
print(f"Total patients in Folder: {len(folder_patients)}")
print(f"Total patients in both: {len(both)}")


Patients only in CSV: 0
Patients only in Folder: 0

Summary:
Total patients in CSV: 642
Total patients in Folder: 642
Total patients in both: 642


In [ ]:
file_path = r"Dataset\CSV\paired_data_datscan.csv"

data = pd.read_csv(file_path)

# Count unique patient IDs in the "PATNO" column
unique_patients = data["PATNO"].nunique()

print(f"Number of unique patients: {unique_patients}")

Number of unique patients: 642


#### Image-level consistency

In [7]:
# Get image IDs from CSV
csv_images = set(df['Image Data ID'].astype(str).unique())

# Get image IDs from folder
folder_images = set()
for patient_folder in os.listdir(images_root):
    patient_path = os.path.join(images_root, patient_folder)
    if os.path.isdir(patient_path):
        for file_name in os.listdir(patient_path):
            if file_name.endswith(".nii") or file_name.endswith(".nii.gz"):
                image_id = os.path.splitext(file_name)[0]
                if image_id.endswith(".nii"):  # Handles .nii.gz double extension
                    image_id = os.path.splitext(image_id)[0]
                folder_images.add(image_id)

# Compare
only_in_csv = csv_images - folder_images
only_in_folder = folder_images - csv_images
both = csv_images & folder_images

print("Images only in CSV:", len(only_in_csv))
print("Images only in Folder:", len(only_in_folder))

if only_in_csv:
    print("Missing in folder:", only_in_csv)
if only_in_folder:
    print("Missing in CSV:", only_in_folder)


print("\nSummary:")
print(f"Total images in CSV: {len(csv_images)}")
print(f"Total images in Folder: {len(folder_images)}")
print(f"Total images in both: {len(both)}")


Images only in CSV: 0
Images only in Folder: 0

Summary:
Total images in CSV: 986
Total images in Folder: 986
Total images in both: 986


## Merge Both Data Types (Assesments + MRI_T2)

### for csv

In [20]:
assessments_df = pd.read_csv("Dataset/CSV/assesments_data_v2.csv")
mri_t2_df = pd.read_csv("Dataset/mri_t2_images/MRI_T2_dataset.csv")


# Rename columns in DaTscan to match those in assessments
mri_t2_df = mri_t2_df.rename(columns={"Subject": "PATNO", "Visit": "EVENT_ID"})


# Merge on both keys, keeping only matches in both dataframes
merged_df = pd.merge(
    assessments_df,
    mri_t2_df[["PATNO", "EVENT_ID", "Image Data ID"]],
    on=["PATNO", "EVENT_ID"],
    how="inner"
)


# Move "Image Data ID" to be 3rd column
if "Image Data ID" in merged_df.columns:
    cols = merged_df.columns.tolist()
    cols.remove("Image Data ID")
    cols.insert(2, "Image Data ID")
    merged_df = merged_df[cols]


# Save the merged dataset
merged_csv_path = "Dataset/CSV/paired_data_v2_mri_t2.csv"
merged_df.to_csv(merged_csv_path, index=False)

print(f"Original dataset shape: {assessments_df.shape}")
print(f"Merged dataset shape: {merged_df.shape}")

Original dataset shape: (1972, 114)
Merged dataset shape: (130, 115)


### for MRI_T2 folder

In [21]:
images_dir = "Dataset/mri_t2_images/Resized_mri_t2(not paired)"

merged_df = pd.read_csv(merged_csv_path)


# Create mapping: patient -> set of allowed image IDs
patient_to_images = (
    merged_df.groupby("PATNO")["Image Data ID"]
    .apply(lambda x: set(x.astype(str))) # convert image_id to string for comparaison
    .to_dict()
)

removed_patients_count = 0
removed_images_count = 0

# Go through each patient folder
for patient_folder in os.listdir(images_dir):
    patient_path = os.path.join(images_dir, patient_folder)
    if not os.path.isdir(patient_path):
        continue  # Skip if not a folder

    patient_id = str(patient_folder)

    # Case 1: Patient not in CSV → delete whole folder
    if patient_id not in map(str, patient_to_images.keys()):
        shutil.rmtree(patient_path)
        removed_patients_count += 1
        print(f"🗑 Deleted entire folder for patient {patient_folder}")
        continue
    
    # Case 2: Patient exists → check their images
    allowed_images = patient_to_images[int(patient_id)]

    for file_name in os.listdir(patient_path):
        if not file_name.endswith(".nii"):
            continue

        image_id = file_name.replace(".nii", "")
        if image_id not in allowed_images:
            os.remove(os.path.join(patient_path, file_name))
            removed_images_count += 1
            print(f"🗑 Removed image {image_id} from patient {patient_folder}")


# Rename folder after cleanup
os.rename(images_dir, "Dataset/mri_t2_images/Paired_data_v2_mri_t2")

# ==== FINAL SUMMARY ====
print("\n" + "="*60)
print(f"✅ Cropped images folder synced with merged dataset")
print(f"   Deleted {removed_patients_count} entire patient folders")
print(f"   Deleted {removed_images_count} individual images")


🗑 Deleted entire folder for patient 3104
🗑 Deleted entire folder for patient 3105
🗑 Removed image I304299 from patient 3107
🗑 Removed image I304310 from patient 3107
🗑 Removed image I330465 from patient 3107
🗑 Deleted entire folder for patient 3108
🗑 Deleted entire folder for patient 3111
🗑 Deleted entire folder for patient 3112
🗑 Deleted entire folder for patient 3113
🗑 Deleted entire folder for patient 3114
🗑 Deleted entire folder for patient 3115
🗑 Deleted entire folder for patient 3116
🗑 Removed image I330607 from patient 3118
🗑 Removed image I333099 from patient 3118
🗑 Removed image I371310 from patient 3118
🗑 Removed image I406789 from patient 3118
🗑 Removed image I407152 from patient 3118
🗑 Deleted entire folder for patient 3119
🗑 Removed image I405839 from patient 3123
🗑 Removed image I405939 from patient 3123
🗑 Deleted entire folder for patient 3126
🗑 Deleted entire folder for patient 3127
🗑 Removed image I406338 from patient 3130
🗑 Removed image I407365 from patient 3130
🗑 Re

### Verification

#### Patient-level consistency

In [22]:
# Paths
csv_path = "Dataset/CSV/paired_data_v2_mri_t2.csv"
images_root = "Dataset/mri_t2_images/Paired_data_v2_mri_t2"

# --- Get patient IDs from CSV ---
df = pd.read_csv(csv_path)
csv_patients = set(df['PATNO'].astype(str).unique())

# --- Get patient IDs from image folders ---
folder_patients = set(os.listdir(images_root))

# --- Compare ---
only_in_csv = csv_patients - folder_patients
only_in_folder = folder_patients - csv_patients
both = csv_patients & folder_patients

print("Patients only in CSV:", len(only_in_csv))
print("Patients only in MRI Folder:", len(only_in_folder))

if only_in_csv:
    print("Missing in MRI Folder:", only_in_csv)
if only_in_folder:
    print("Missing in CSV:", only_in_folder)

# --- Count images for patients in both ---
total_images = 0
for patient_id in both:
    patient_folder = os.path.join(images_root, patient_id)
    total_images += len([f for f in os.listdir(patient_folder) if f.endswith(".nii") or f.endswith(".nii.gz")])


print("\nSummary:")
print(f"Total patients in CSV: {len(csv_patients)}")
print(f"Total patients in MRI Folder: {len(folder_patients)}")
print(f"Total patients in both: {len(both)}")


Patients only in CSV: 0
Patients only in MRI Folder: 0

Summary:
Total patients in CSV: 54
Total patients in MRI Folder: 54
Total patients in both: 54


#### Image-level consistency

In [23]:
# --- Get image IDs from CSV ---
csv_images = set(df['Image Data ID'].astype(str).unique())

# --- Get image IDs from folder ---
folder_images = set()
for patient_folder in os.listdir(images_root):
    patient_path = os.path.join(images_root, patient_folder)
    if os.path.isdir(patient_path):
        for file_name in os.listdir(patient_path):
            if file_name.endswith(".nii") or file_name.endswith(".nii.gz"):
                image_id = os.path.splitext(file_name)[0]
                if image_id.endswith(".nii"):  # Handles .nii.gz double extension
                    image_id = os.path.splitext(image_id)[0]
                folder_images.add(image_id)

# --- Compare ---
only_in_csv = csv_images - folder_images
only_in_folder = folder_images - csv_images
both = csv_images & folder_images

print("Images only in CSV:", len(only_in_csv))
print("Images only in MRI Folder:", len(only_in_folder))

if only_in_csv:
    print("Missing in MRI folder:", only_in_csv)
if only_in_folder:
    print("Missing in CSV:", only_in_folder)


print("\nSummary:")
print(f"Total images in CSV: {len(csv_images)}")
print(f"Total images in MRI Folder: {len(folder_images)}")
print(f"Total images in both: {len(both)}")


Images only in CSV: 0
Images only in MRI Folder: 0

Summary:
Total images in CSV: 130
Total images in MRI Folder: 130
Total images in both: 130


## Merge 3 Data Types (Assesments + DaTscan + MRI_T2)

### for csv

In [3]:
# Load CSVs
assessments_df = pd.read_csv("Dataset/CSV/assesments_data.csv")
DaTscan_df    = pd.read_csv("Dataset/datscan_images/DaTscan_dataset.csv")
MRI_df        = pd.read_csv("Dataset/mri_t2_images/MRI_T2_dataset.csv")


# Clean corrupted DaTscan entries
corrupted_entries = [
    # failed in Conversion
    {"Image Data ID": "I770243",  "Subject": 3107},
    {"Image Data ID": "I1317564", "Subject": 3179},
    # failed in Denoising
    {"Image Data ID": "I1317554", "Subject": 3372},
    {"Image Data ID": "I449046",  "Subject": 4092},
    {"Image Data ID": "I1273880",  "Subject": 50028},
    # failed in cropping
    {"Image Data ID": "I1317553",  "Subject": 3078},
    {"Image Data ID": "I1317557",  "Subject": 3378}
]

for entry in corrupted_entries:
    DaTscan_df = DaTscan_df[~(
        (DaTscan_df["Image Data ID"] == entry["Image Data ID"]) &
        (DaTscan_df["Subject"] == entry["Subject"])
    )]

# Harmonize column names
DaTscan_df = DaTscan_df.rename(columns={"Subject": "PATNO", "Visit": "EVENT_ID", "Image Data ID": "DaTscan_ID"})
MRI_df     = MRI_df.rename(columns={"Subject": "PATNO", "Visit": "EVENT_ID", "Image Data ID": "MRI_T2_ID"})


# Step 1: Merge assessments with DaTscan on PATNO and EVENT_ID (inner join)
merged_df = pd.merge(
    assessments_df, 
    DaTscan_df[["PATNO", "EVENT_ID", "DaTscan_ID"]], 
    on=['PATNO', 'EVENT_ID'], 
    how='inner'
)

# Step 2: Merge the result with MRI on PATNO and EVENT_ID (inner join)
merged_df = pd.merge(
    merged_df, 
    MRI_df[["PATNO", "EVENT_ID", "MRI_T2_ID"]], 
    on=['PATNO', 'EVENT_ID'], 
    how='inner'
)

# Step 3: Remove any rows with missing/empty values in ANY column
# merged_df = merged_df.dropna()


# Reorder the Image IDs columns
cols = merged_df.columns.tolist()
for col in ["DaTscan_ID", "MRI_T2_ID"]:
    cols.remove(col)
cols.insert(2, "DaTscan_ID")
cols.insert(3, "MRI_T2_ID")
merged_df = merged_df[cols]


# Save merged dataset
merged_csv_path = "Dataset/CSV/3_paired_data.csv"
merged_df.to_csv(merged_csv_path, index=False)

print(f"Original assessments shape: {assessments_df.shape}")
print(f"DaTscan_csv shape:    {DaTscan_df.shape}")
print(f"MRI_T2_csv shape:        {MRI_df.shape}")
print(f"Merged dataset shape:      {merged_df.shape}")

Original assessments shape: (4097, 84)
DaTscan_csv shape:    (3832, 12)
MRI_T2_csv shape:        (721, 12)
Merged dataset shape:      (167, 86)


In [4]:
# Load CSVs
assessments_df = pd.read_csv("Dataset/CSV/assesments_data_v2.csv")
DaTscan_df    = pd.read_csv("Dataset/datscan_images/DaTscan_dataset.csv")
MRI_df        = pd.read_csv("Dataset/mri_t2_images/MRI_T2_dataset.csv")


# Clean corrupted DaTscan entries
corrupted_entries = [
    # failed in Conversion
    {"Image Data ID": "I770243",  "Subject": 3107},
    {"Image Data ID": "I1317564", "Subject": 3179},
    # failed in Denoising
    {"Image Data ID": "I1317554", "Subject": 3372},
    {"Image Data ID": "I449046",  "Subject": 4092},
    {"Image Data ID": "I1273880",  "Subject": 50028},
    # failed in cropping
    {"Image Data ID": "I1317553",  "Subject": 3078},
    {"Image Data ID": "I1317557",  "Subject": 3378}
]

for entry in corrupted_entries:
    DaTscan_df = DaTscan_df[~(
        (DaTscan_df["Image Data ID"] == entry["Image Data ID"]) &
        (DaTscan_df["Subject"] == entry["Subject"])
    )]

# Harmonize column names
DaTscan_df = DaTscan_df.rename(columns={"Subject": "PATNO", "Visit": "EVENT_ID", "Image Data ID": "DaTscan_ID"})
MRI_df     = MRI_df.rename(columns={"Subject": "PATNO", "Visit": "EVENT_ID", "Image Data ID": "MRI_T2_ID"})


# Step 1: Merge assessments with DaTscan on PATNO and EVENT_ID (inner join)
merged_df = pd.merge(
    assessments_df, 
    DaTscan_df[["PATNO", "EVENT_ID", "DaTscan_ID"]], 
    on=['PATNO', 'EVENT_ID'], 
    how='inner'
)

# Step 2: Merge the result with MRI on PATNO and EVENT_ID (inner join)
merged_df = pd.merge(
    merged_df, 
    MRI_df[["PATNO", "EVENT_ID", "MRI_T2_ID"]], 
    on=['PATNO', 'EVENT_ID'], 
    how='inner'
)

# Step 3: Remove any rows with missing/empty values in ANY column
# merged_df = merged_df.dropna()


# Reorder the Image IDs columns
cols = merged_df.columns.tolist()
for col in ["DaTscan_ID", "MRI_T2_ID"]:
    cols.remove(col)
cols.insert(2, "DaTscan_ID")
cols.insert(3, "MRI_T2_ID")
merged_df = merged_df[cols]


# Save merged dataset
merged_csv_path = "Dataset/CSV/3_paired_data_v2.csv"
merged_df.to_csv(merged_csv_path, index=False)

print(f"Original assessments shape: {assessments_df.shape}")
print(f"DaTscan_csv shape:    {DaTscan_df.shape}")
print(f"MRI_T2_csv shape:        {MRI_df.shape}")
print(f"Merged dataset shape:      {merged_df.shape}")

Original assessments shape: (1972, 114)
DaTscan_csv shape:    (3832, 12)
MRI_T2_csv shape:        (721, 12)
Merged dataset shape:      (98, 116)


In [5]:
# Check missing values before
missing_before = merged_df.isna().sum().sum()
print(f"Missing values: {missing_before}")

# Imputers
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Store column names and index before transformation
columns = merged_df.columns
index = merged_df.index

# Apply imputation (returns numpy array)
imputed_array = categorical_imputer.fit_transform(merged_df)

# Convert back to DataFrame with original column names and index
merged_df = pd.DataFrame(imputed_array, columns=columns, index=index)

# Print missing values after imputation
missing_after = merged_df.isna().sum().sum()
print(f"Missing values after: {missing_after}")

# Save the imputed data back to the original CSV file
merged_csv_path = "Dataset/CSV/3_paired_data_v2.csv"
merged_df.to_csv(merged_csv_path, index=False)
print(f"\nImputed data saved to: {merged_csv_path}")

Missing values: 114
Missing values after: 0

Imputed data saved to: Dataset/CSV/3_paired_data_v2.csv


### for DaTscan+MRI_T2 folder

In [ ]:
# ===============================
# Sync with image folders
# ===============================
dat_images_dir = "Dataset/datscan_images/Cropped_NIfTI"
mri_images_dir = "Dataset/mri_t2_images/PPMI_mri_t2_flat"

# Create mapping: patient → allowed DaTscan IDs
patient_to_dat_images = (
    merged_df.groupby("PATNO")["DaTscan_ID"]
    .apply(lambda x: set(x.astype(str)))
    .to_dict()
)

# Create mapping: patient → allowed MRI IDs
patient_to_mri_images = (
    merged_df.groupby("PATNO")["MRI_T2_ID"]
    .apply(lambda x: set(x.astype(str)))
    .to_dict()
)

removed_patients_dat = 0
removed_images_dat   = 0
removed_patients_mri = 0
removed_images_mri   = 0

# ---- Cleanup DaTscan ----
for patient_folder in os.listdir(dat_images_dir):
    patient_path = os.path.join(dat_images_dir, patient_folder)
    if not os.path.isdir(patient_path):
        continue

    patient_id = str(patient_folder)

    if patient_id not in map(str, patient_to_dat_images.keys()):
        shutil.rmtree(patient_path)
        removed_patients_dat += 1
        print(f"🗑 Deleted entire DaTscan folder for patient {patient_folder}")
        continue

    allowed_images = patient_to_dat_images[int(patient_id)]
    for file_name in os.listdir(patient_path):
        if not file_name.endswith(".nii.gz"):
            continue

        image_id = file_name.replace(".nii.gz", "")
        if image_id not in allowed_images:
            os.remove(os.path.join(patient_path, file_name))
            removed_images_dat += 1
            print(f"🗑 Removed DaTscan {image_id} from patient {patient_folder}")

# ---- Cleanup MRI ----
for patient_folder in os.listdir(mri_images_dir):
    patient_path = os.path.join(mri_images_dir, patient_folder)
    if not os.path.isdir(patient_path):
        continue
    patient_id = str(patient_folder)

    if patient_id not in map(str, patient_to_mri_images.keys()):
        shutil.rmtree(patient_path)
        removed_patients_mri += 1
        print(f"🗑 Deleted entire MRI folder for patient {patient_folder}")
        continue

    allowed_images = patient_to_mri_images[int(patient_id)]
    for file_name in os.listdir(patient_path):
        if not file_name.endswith(".nii"):
            continue

        image_id = file_name.replace(".nii", "")
        if image_id not in allowed_images:
            os.remove(os.path.join(patient_path, file_name))
            removed_images_mri += 1
            print(f"🗑 Removed MRI {image_id} from patient {patient_folder}")


# Final Renaming 
os.rename(dat_images_dir, "Dataset/datscan_images/3_Paired_DaTscan")
os.rename(mri_images_dir, "Dataset/mri_t2_images/3_Paired_MRI_T2")

# ==== FINAL SUMMARY ====
print("\n" + "="*60)
print(f"✅ Images synced with merged dataset")
print(f"   Deleted {removed_patients_dat} DaTscan patient folders")
print(f"   Deleted {removed_images_dat} DaTscan individual images")
print(f"   Deleted {removed_patients_mri} MRI patient folders")
print(f"   Deleted {removed_images_mri} MRI individual images")


🗑 Deleted entire DaTscan folder for patient 100001
🗑 Deleted entire DaTscan folder for patient 100002
🗑 Deleted entire DaTscan folder for patient 100004
🗑 Deleted entire DaTscan folder for patient 100005
🗑 Deleted entire DaTscan folder for patient 100006
🗑 Deleted entire DaTscan folder for patient 100007
🗑 Deleted entire DaTscan folder for patient 100012
🗑 Deleted entire DaTscan folder for patient 100017
🗑 Deleted entire DaTscan folder for patient 100018
🗑 Deleted entire DaTscan folder for patient 100267
🗑 Deleted entire DaTscan folder for patient 100268
🗑 Deleted entire DaTscan folder for patient 100639
🗑 Deleted entire DaTscan folder for patient 100738
🗑 Deleted entire DaTscan folder for patient 100842
🗑 Deleted entire DaTscan folder for patient 100878
🗑 Deleted entire DaTscan folder for patient 100889
🗑 Deleted entire DaTscan folder for patient 100890
🗑 Deleted entire DaTscan folder for patient 100891
🗑 Deleted entire DaTscan folder for patient 100898
🗑 Deleted entire DaTscan folder

### Verification

#### Patient-level consistency

In [7]:
# Paths
csv_path = "Dataset/CSV/3_paired_data.csv"
dat_images_root = "Dataset/datscan_images/3_Paired_DaTscan"
mri_images_root = "Dataset/mri_t2_images/3_Paired_MRI_T2"

# --- Get patient IDs from merged CSV ---
df = pd.read_csv(csv_path)
csv_patients = set(df['PATNO'].astype(str).unique())

# --- Get patient IDs from DaTscan folders ---
dat_patients = set(os.listdir(dat_images_root))

# --- Get patient IDs from MRI folders ---
mri_patients = set(os.listdir(mri_images_root))

# --- Compare ---
only_in_csv  = csv_patients - dat_patients - mri_patients
only_in_dat  = dat_patients - csv_patients
only_in_mri  = mri_patients - csv_patients
common_all   = csv_patients & dat_patients & mri_patients

print("Patients only in CSV:", len(only_in_csv))
print("Patients only in DaTscan folder:", len(only_in_dat))
print("Patients only in MRI folder:", len(only_in_mri))

print("\nSummary:")
print(f"Total patients in CSV: {len(csv_patients)}")
print(f"Total patients in DaTscan folder: {len(dat_patients)}")
print(f"Total patients in MRI folder: {len(mri_patients)}")
print(f"Total patients present in all 3: {len(common_all)}")

Patients only in CSV: 18
Patients only in DaTscan folder: 0
Patients only in MRI folder: 0

Summary:
Total patients in CSV: 60
Total patients in DaTscan folder: 42
Total patients in MRI folder: 42
Total patients present in all 3: 42


#### Image-level consistency

In [8]:
# Paths
csv_path = "Dataset/CSV/3_paired_data.csv"
dat_images_root = "Dataset/datscan_images/3_Paired_DaTscan"
mri_images_root = "Dataset/mri_t2_images/3_Paired_MRI_T2"


# --- Extract image IDs ---
df = pd.read_csv(csv_path)
csv_dat_images = set(df['DaTscan_ID'].astype(str).unique())
csv_mri_images = set(df['MRI_T2_ID'].astype(str).unique())

# --- Collect actual DaTscan images from folders ---
folder_dat_images = set()
for patient_folder in os.listdir(dat_images_root):
    patient_path = os.path.join(dat_images_root, patient_folder)
    if os.path.isdir(patient_path):
        for file_name in os.listdir(patient_path):
            if file_name.endswith(".nii") or file_name.endswith(".nii.gz"):
                image_id = os.path.splitext(file_name)[0]
                if image_id.endswith(".nii"):  # handle .nii.gz
                    image_id = os.path.splitext(image_id)[0]
                folder_dat_images.add(image_id)

# --- Collect actual MRI images from folders ---
folder_mri_images = set()
for patient_folder in os.listdir(mri_images_root):
    patient_path = os.path.join(mri_images_root, patient_folder)
    if os.path.isdir(patient_path):
        for file_name in os.listdir(patient_path):
            if file_name.endswith(".nii") or file_name.endswith(".nii.gz"):
                image_id = os.path.splitext(file_name)[0]
                if image_id.endswith(".nii"):  # handle .nii.gz
                    image_id = os.path.splitext(image_id)[0]
                folder_mri_images.add(image_id)

# --- Compare ---
only_in_csv_dat = csv_dat_images - folder_dat_images
only_in_folder_dat = folder_dat_images - csv_dat_images
both_dat = csv_dat_images & folder_dat_images

only_in_csv_mri = csv_mri_images - folder_mri_images
only_in_folder_mri = folder_mri_images - csv_mri_images
both_mri = csv_mri_images & folder_mri_images

print("=== DATSCAN ===")
print("Images only in CSV:", len(only_in_csv_dat))
print("Images only in Folder:", len(only_in_folder_dat))
print("\nSummary:")
print(f"Total images in CSV: {len(csv_dat_images)}")
print(f"Total images in Folder: {len(folder_dat_images)}")
print(f"Total images in both: {len(both_dat)}")

print("\n=== MRI T2 ===")
print("Images only in CSV:", len(only_in_csv_mri))
print("Images only in Folder:", len(only_in_folder_mri))
print("\nSummary:")
print(f"Total images in CSV: {len(csv_mri_images)}")
print(f"Total images in Folder: {len(folder_mri_images)}")
print(f"Total images in both: {len(both_mri)}")

=== DATSCAN ===
Images only in CSV: 32
Images only in Folder: 0

Summary:
Total images in CSV: 81
Total images in Folder: 49
Total images in both: 49

=== MRI T2 ===
Images only in CSV: 69
Images only in Folder: 0

Summary:
Total images in CSV: 167
Total images in Folder: 98
Total images in both: 98


#### General 

In [ ]:
# ===============================
# VERIFICATION CODE
# ===============================

# Load the merged CSV
merged_csv_path = "Dataset/CSV/3_paired_data_v2.csv"
merged_df = pd.read_csv(merged_csv_path)

# Define directories
dat_images_dir = "Dataset/datscan_images/3_Paired_DaTscan"
mri_images_dir = "Dataset/mri_t2_images/3_Paired_MRI_T2"

print("=" * 80)
print("DATA PAIRING VERIFICATION")
print("=" * 80)

# ---- CSV Statistics ----
csv_total_rows = len(merged_df)
csv_unique_patients = merged_df["PATNO"].nunique()
csv_datscan_ids = set(merged_df["DaTscan_ID"].astype(str))
csv_mri_ids = set(merged_df["MRI_T2_ID"].astype(str))

print(f"\n📊 CSV Statistics:")
print(f"   Total rows (records): {csv_total_rows}")
print(f"   Unique patients (PATNO): {csv_unique_patients}")
print(f"   Unique DaTscan IDs: {len(csv_datscan_ids)}")
print(f"   Unique MRI IDs: {len(csv_mri_ids)}")


# ---- DaTscan Directory Statistics ----
dat_patient_folders = [f for f in os.listdir(dat_images_dir) if os.path.isdir(os.path.join(dat_images_dir, f))]
dat_total_images = 0
dat_image_ids = set()

for patient_folder in dat_patient_folders:
    patient_path = os.path.join(dat_images_dir, patient_folder)
    for file_name in os.listdir(patient_path):
        if file_name.endswith(".nii.gz"):
            dat_total_images += 1
            image_id = file_name.replace(".nii.gz", "")
            dat_image_ids.add(image_id)

print(f"\n🧠 DaTscan Directory Statistics:")
print(f"   Patient folders: {len(dat_patient_folders)}")
print(f"   Total images (.nii.gz): {dat_total_images}")
print(f"   Unique image IDs: {len(dat_image_ids)}")


# ---- MRI Directory Statistics ----
mri_patient_folders = [f for f in os.listdir(mri_images_dir) if os.path.isdir(os.path.join(mri_images_dir, f))]
mri_total_images = 0
mri_image_ids = set()

for patient_folder in mri_patient_folders:
    patient_path = os.path.join(mri_images_dir, patient_folder)
    for file_name in os.listdir(patient_path):
        if file_name.endswith(".nii") or file_name.endswith(".nii.gz"):
            mri_total_images += 1
            image_id = file_name.replace(".nii.gz", "").replace(".nii", "")
            mri_image_ids.add(image_id)

print(f"\n🧠 MRI Directory Statistics:")
print(f"   Patient folders: {len(mri_patient_folders)}")
print(f"   Total images (.nii/.nii.gz): {mri_total_images}")
print(f"   Unique image IDs: {len(mri_image_ids)}")


# ===============================
# VERIFICATION CHECKS
# ===============================
print(f"\n{'=' * 80}")
print("VERIFICATION RESULTS")
print("=" * 80)

all_checks_passed = True

# Check 1: Total rows vs total images
print(f"\n✓ Check 1: Total Counts Match")
print(f"   CSV rows: {csv_total_rows}")
print(f"   DaTscan images: {dat_total_images}")
print(f"   MRI images: {mri_total_images}")

if csv_total_rows == dat_total_images == mri_total_images:
    print(f"   ✅ PASSED: All counts match ({csv_total_rows} records)")
else:
    print(f"   ❌ FAILED: Counts don't match!")
    all_checks_passed = False


# Check 2: Unique patients vs patient folders
print(f"\n✓ Check 2: Patient Folder Counts Match")
print(f"   CSV unique patients: {csv_unique_patients}")
print(f"   DaTscan folders: {len(dat_patient_folders)}")
print(f"   MRI folders: {len(mri_patient_folders)}")

if csv_unique_patients == len(dat_patient_folders) == len(mri_patient_folders):
    print(f"   ✅ PASSED: All patient counts match ({csv_unique_patients} patients)")
else:
    print(f"   ❌ FAILED: Patient counts don't match!")
    all_checks_passed = False


# Check 3: All CSV DaTscan IDs exist in file system
print(f"\n✓ Check 3: All CSV DaTscan IDs Have Corresponding Files")
missing_datscan = csv_datscan_ids - dat_image_ids
if len(missing_datscan) == 0:
    print(f"   ✅ PASSED: All {len(csv_datscan_ids)} DaTscan IDs from CSV exist as files")
else:
    print(f"   ❌ FAILED: {len(missing_datscan)} DaTscan IDs missing from file system:")
    for missing_id in list(missing_datscan)[:5]:  # Show first 5
        print(f"      - {missing_id}")
    if len(missing_datscan) > 5:
        print(f"      ... and {len(missing_datscan) - 5} more")
    all_checks_passed = False


# Check 4: All CSV MRI IDs exist in file system
print(f"\n✓ Check 4: All CSV MRI IDs Have Corresponding Files")
missing_mri = csv_mri_ids - mri_image_ids
if len(missing_mri) == 0:
    print(f"   ✅ PASSED: All {len(csv_mri_ids)} MRI IDs from CSV exist as files")
else:
    print(f"   ❌ FAILED: {len(missing_mri)} MRI IDs missing from file system:")
    for missing_id in list(missing_mri)[:5]:  # Show first 5
        print(f"      - {missing_id}")
    if len(missing_mri) > 5:
        print(f"      ... and {len(missing_mri) - 5} more")
    all_checks_passed = False


# Check 5: No extra files in directories (not in CSV)
print(f"\n✓ Check 5: No Extra Files in Directories")
extra_datscan = dat_image_ids - csv_datscan_ids
extra_mri = mri_image_ids - csv_mri_ids

if len(extra_datscan) == 0 and len(extra_mri) == 0:
    print(f"   ✅ PASSED: No extra files found")
else:
    if len(extra_datscan) > 0:
        print(f"   ⚠️  WARNING: {len(extra_datscan)} extra DaTscan files not in CSV")
        for extra_id in list(extra_datscan)[:3]:
            print(f"      - {extra_id}")
    if len(extra_mri) > 0:
        print(f"   ⚠️  WARNING: {len(extra_mri)} extra MRI files not in CSV")
        for extra_id in list(extra_mri)[:3]:
            print(f"      - {extra_id}")


# Check 6: Each CSV row has both images
print(f"\n✓ Check 6: Every CSV Row Has Both DaTscan and MRI Images")
rows_with_missing_images = 0
for idx, row in merged_df.iterrows():
    datscan_id = str(row["DaTscan_ID"])
    mri_id = str(row["MRI_T2_ID"])
    
    if datscan_id not in dat_image_ids or mri_id not in mri_image_ids:
        rows_with_missing_images += 1
        if rows_with_missing_images <= 3:  # Show first 3 examples
            print(f"   ❌ Row {idx}: PATNO={row['PATNO']}, DaTscan={datscan_id}, MRI={mri_id}")

if rows_with_missing_images == 0:
    print(f"   ✅ PASSED: All {csv_total_rows} rows have both image types")
else:
    print(f"   ❌ FAILED: {rows_with_missing_images} rows missing one or both images")
    all_checks_passed = False


# ===============================
# FINAL SUMMARY
# ===============================
print(f"\n{'=' * 80}")
if all_checks_passed:
    print("✅ ALL VERIFICATION CHECKS PASSED!")
    print(f"✅ Data is correctly paired: {csv_total_rows} complete records with all 3 data types")
    print(f"✅ {csv_unique_patients} patients have complete DaTscan + MRI + CSV data")
else:
    print("❌ VERIFICATION FAILED - Data pairing issues detected")
    print("⚠️  Please review the issues above and re-run the sync script")
print("=" * 80)


DATA PAIRING VERIFICATION

📊 CSV Statistics:
   Total rows (records): 98
   Unique patients (PATNO): 42
   Unique DaTscan IDs: 49
   Unique MRI IDs: 98

🧠 DaTscan Directory Statistics:
   Patient folders: 42
   Total images (.nii.gz): 49
   Unique image IDs: 49

🧠 MRI Directory Statistics:
   Patient folders: 42
   Total images (.nii/.nii.gz): 98
   Unique image IDs: 98

VERIFICATION RESULTS

✓ Check 1: Total Counts Match
   CSV rows: 98
   DaTscan images: 49
   MRI images: 98
   ❌ FAILED: Counts don't match!

✓ Check 2: Patient Folder Counts Match
   CSV unique patients: 42
   DaTscan folders: 42
   MRI folders: 42
   ✅ PASSED: All patient counts match (42 patients)

✓ Check 3: All CSV DaTscan IDs Have Corresponding Files
   ✅ PASSED: All 49 DaTscan IDs from CSV exist as files

✓ Check 4: All CSV MRI IDs Have Corresponding Files
   ✅ PASSED: All 98 MRI IDs from CSV exist as files

✓ Check 5: No Extra Files in Directories
   ✅ PASSED: No extra files found

✓ Check 6: Every CSV Row Has 

### Merging DaTscan + MRI in one folder

In [3]:
# === Paths ===
datscan_path = r"Dataset\datscan_images\3_Paired_DaTscan"
mri_path = r"Dataset\mri_t2_images\Resized_mri_t2"
merged_path = r"Dataset\Merged_DaTscan_MRI"

os.makedirs(merged_path, exist_ok=True)

# === Helper: compress MRI .nii to .nii.gz ===
def compress_nii_to_gz(input_file, output_file):
    img = nib.load(input_file)
    nib.save(img, output_file)  # nibabel handles compression automatically

# === Iterate through patient folders ===
patients = sorted(os.listdir(datscan_path))
total_patients = len(patients)
processed = 0
failed_patients = []

for patient_id in patients:
    processed += 1
    print(f"\n[{processed}/{total_patients}] Merging patient: {patient_id}")

    datscan_patient_dir = os.path.join(datscan_path, patient_id)
    mri_patient_dir = os.path.join(mri_path, patient_id)

    # skip if not a directory
    if not os.path.isdir(datscan_patient_dir):
        continue

    # Output folder for this patient
    merged_patient_dir = os.path.join(merged_path, patient_id)
    os.makedirs(merged_patient_dir, exist_ok=True)

    try:
        # --- Copy DaTscan images directly ---
        for file in os.listdir(datscan_patient_dir):
            if file.endswith(".nii") or file.endswith(".nii.gz"):
                src = os.path.join(datscan_patient_dir, file)
                dst = os.path.join(merged_patient_dir, file)
                shutil.copy2(src, dst)

        # --- Copy and compress MRI images ---
        if os.path.exists(mri_patient_dir):
            for file in os.listdir(mri_patient_dir):
                if file.endswith(".nii") or file.endswith(".nii.gz"):
                    src = os.path.join(mri_patient_dir, file)
                    dst = os.path.join(merged_patient_dir, file)

                    # compress to .nii.gz if needed
                    if file.endswith(".nii"):
                        dst += ".gz"
                        compress_nii_to_gz(src, dst)
                    else:
                        shutil.copy2(src, dst)
        else:
            print(f"⚠ No MRI folder for patient {patient_id}")

    except Exception as e:
        print(f"[✘] Failed to merge patient {patient_id}: {e}")
        failed_patients.append(patient_id)

# === Final Summary ===
print("\n" + "=" * 60)
print("✅ Merge complete!")
print(f"Total patients processed: {processed}")
if failed_patients:
    print(f"⚠ Failed patients ({len(failed_patients)}): {failed_patients}")
else:
    print("All patients merged successfully!")



[1/42] Merging patient: 3107

[2/42] Merging patient: 3118

[3/42] Merging patient: 3120

[4/42] Merging patient: 3122

[5/42] Merging patient: 3123

[6/42] Merging patient: 3124

[7/42] Merging patient: 3128

[8/42] Merging patient: 3130

[9/42] Merging patient: 3131

[10/42] Merging patient: 3132

[11/42] Merging patient: 3176

[12/42] Merging patient: 3178

[13/42] Merging patient: 3181

[14/42] Merging patient: 3185

[15/42] Merging patient: 3190

[16/42] Merging patient: 3307

[17/42] Merging patient: 3309

[18/42] Merging patient: 3321

[19/42] Merging patient: 3323

[20/42] Merging patient: 3325

[21/42] Merging patient: 3328

[22/42] Merging patient: 3360

[23/42] Merging patient: 3375

[24/42] Merging patient: 3378

[25/42] Merging patient: 3383

[26/42] Merging patient: 3567

[27/42] Merging patient: 3585

[28/42] Merging patient: 3586

[29/42] Merging patient: 3587

[30/42] Merging patient: 3588

[31/42] Merging patient: 3593

[32/42] Merging patient: 3789

[33/42] Merging 